In [ ]:
#import ultralytics
#ultralytics.checks()
#from ultralytics import YOLO

import yolo
from yolo import YOLO

from PIL import Image
import torch
import yaml

import os
import random
#from ultralytics.utils.plotting import plot_labels
import pandas as pd
import sys

module_path = os.path.abspath(os.path.join('..'))
print(module_path)
module_path = module_path+'/data_preprocessing'
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)

#import visualization_utils as visutils

In [ ]:
device = "0" if torch.cuda.is_available() else "cpu"
if device == "0":
    torch.cuda.set_device(0) # Set to your desired GPU number

print(device)

In [ ]:
# Create a new YOLO model from scratch
#model = YOLO('yolov8n.yaml')

# Load a pretrained YOLO model (recommended for training) - for now, we keep the small version
#pretrained_model_name = 'pfeifer_penguins_poland_10percentbckgd_yolov8m_120epoch'
#model_path = 'runs/detect/' + pretrained_model_name + '/weights/best.pt'
#model = YOLO('yolov8m.pt', task='da_detect')
#model = YOLO("yolov8.yaml").load("yolov8m.pt")
#model = YOLO('yolov8m.pt', task='da_detect')
#model = YOLO('yolov8m.pt', task='detect')
#model = YOLO("yolov8m.yaml", task='detect').load("yolov8m.pt")
model = YOLO("yolov8m.yaml", task='detect', subtask='multidomainclassifier_multisources').load("yolov8m.pt")


#model.load_weights('yolov8m.pt')


#PRETRAINED_MODEL_NAME = 'pfeifer_penguins_poland_10percentbckgd_yolov8m_120epoch'
#PRETRAINED_MODEL_PATH = 'src/model/runs/detect/' + PRETRAINED_MODEL_NAME + '/weights/best.pt'

#MODEL_NAME = 'pfeifer_penguins_poland_palmyra_10percent_bckgd_yolov8m_120epochs'
#TASK = 'detect' # Choose between: 'deepcoral_detect' 'detect'
#model = YOLO('yolov8m.pt')

In [ ]:
print(model.task)
print(model.subtask)

In [ ]:
model.model.model
for i, a in enumerate(model.model.model):
    if i == 24:
        print("block", i)
        print(a)

In [ ]:
# TRAIN the model on our dataset (data.yml config file) 

model_name = 'TEST_features_loss'

results = model.train(
   data='data_.yaml',
   #imgsz=480,  # we are trying with several img size so we do not precise the size -> will automatically resize all images to 640x640
   epochs=2,
   batch=8, #32,
   #cos_lr=True,
   #dropout=0.3,
   #optimizer='Adam',
   patience=10,
   device=0,
   verbose=True,
   val = True,
   #lr0=0.001,
   #lrf=0.0001,
   degrees=90, fliplr=0.5, flipud=0.5, scale=0.5, # augmentation parameters
   name=model_name)

In [ ]:

selected_img = []
img_path = '/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/test/'
selected_img.extend(random.choices(os.listdir(img_path + '/images/'), k=5))

# Predict results for randomly selected images
results = model.predict(
        #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
        source = [os.path.join(img_path + 'images/', img) for img in selected_img],
        conf = 0.1, 
        iou = 0.1,
        show = False,
        save = False
    )

In [ ]:
model.model.model
for i, a in enumerate(model.model.model):
    if i == 23:
        print("block", i)
        print(a)

In [ ]:
model.subtask

In [ ]:
print(B)

In [ ]:

module_path = os.path.abspath(os.path.join('..', '..'))
print(module_path)
module_path = module_path+'/runs'
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
PRETRAINED_MODEL_PATH = module_path + '/detect/YOLO_pe_palmyra_10percentbkgd_test2/weights/best.pt'
model = YOLO('yolov8m.yaml', task='detect').load(PRETRAINED_MODEL_PATH)

In [ ]:
selected_img = []
img_path = '/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/test/'
for subdataset in ['global_birds_penguins', 'global_birds_palmyra']:
    selected_img.extend(random.choices(os.listdir(img_path + subdataset + '/images/'), k=10))

results = model.predict(
        #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
        source = [os.path.join(img_path + 'images/', img) for img in selected_img],
        conf = 0.1, 
        iou = 0.1,
        show = False,
        save = False
    )

In [ ]:
results[5].boxes

## Visualization

In [ ]:
TASK = 'deepcoral_detect' #detect
#model_name = 'deepcoral_test4'
#model_path = 'runs/' + TASK + '/' + model_name + '/weights/best.pt'
#model = YOLO(model_path, TASK)
model_name = 'deepcoral_background_lscale16_epochs20_coralgain10'
PRETRAINED_MODEL_PATH = 'runs/' + TASK + '/' + model_name + '/weights/best.pt'

model = YOLO(PRETRAINED_MODEL_PATH, TASK)

In [ ]:
datasets = ['source', 'target'] #['global_birds_poland', 'global_birds_palmyra', 'global_birds_penguins', 'global_birds_pfeifer']
fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
img_path = data['path'] + '/test/'

In [ ]:
# FOR NORMAL MODEL
# Select randomly 10 images from the test dataset

if TASK == 'detect':
    selected_img = []
    for subdataset in datasets:
        selected_img.extend(random.choices(os.listdir(img_path + subdataset + '/images/'), k=6))

    results = model.predict(
            #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
            source = [os.path.join(img_path + 'images/', img) for img in selected_img],
            conf = 0.2, 
            iou = 0.1,
            show=False,
            save=False
        )

In [ ]:
if TASK == 'detect':
    for img, result in zip(selected_img, results):

        detection_boxes = []
        save_path = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/prediction_' + os.path.basename(result.path).split('.jpg')[0] + '.jpg'
        for detect in range(len(result.boxes.cls)):
            det = {}
            det['conf'] = result.boxes.conf[detect].cpu()
            det['category'] = result.boxes.cls[detect].cpu()
            coords = result.boxes.xywhn[detect].cpu()
            det['bbox'] = [coords[0]-coords[2]/2, coords[1]-coords[3]/2, coords[2], coords[3]]
            detection_boxes.append(det)
            
        im_path = os.path.join(img_path + 'images/', img)
        visutils.draw_bounding_boxes_on_file(im_path, save_path, detection_boxes,
                                        confidence_threshold=0.0, detector_label_map=None,
                                        thickness=1,expansion=0, colormap=['Red'])

        selected_label = img_path + 'labels/' + os.path.basename(result.path).split('.jpg')[0] + '.txt'
        if os.path.exists(selected_label):
            detection_boxes = []
            df = pd.read_csv(selected_label, sep='\t', header=None, index_col=False)
            for irow, row in df.iterrows():  
                det = {}
                det['conf'] = None
                det['category'] = row[0]
                det['bbox'] = [row[1]-row[3]/2, row[2]-row[4]/2, row[3], row[4]]
                detection_boxes.append(det)
        
            # Draw annotations
            save_path2 = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/prediction_label_' + os.path.basename(result.path).split('.hpg')[0] + '.jpg'
            visutils.draw_bounding_boxes_on_file(save_path, save_path2, detection_boxes,
                                            confidence_threshold=0.0, detector_label_map=None,
                                            thickness=1,expansion=0, colormap=['SpringGreen'])
                                            
            # Remove predictions-only images
            os.remove(save_path)


In [ ]:
# FOR DEEP CORAL MODEL
# Select randomly 10 images from the test dataset

if TASK == 'deepcoral_detect':

    for subdataset in datasets:
        selected_img = random.choices(os.listdir(img_path + subdataset + '/images/'), k=12)

        results = model.predict(
                #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
                source = [os.path.join(img_path, subdataset + '/images/', img) for img in selected_img],
                conf = 0.2, 
                iou = 0.1,
                show=False,
                save=False
            )
        
        for img, result in zip(selected_img, results):

            detection_boxes = []
            save_path = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/' + TASK + '/' + model_name + '/prediction_' + os.path.basename(result.path).split('.jpg')[0] + '.jpg'
            for detect in range(len(result.boxes.cls)):
                det = {}
                det['conf'] = result.boxes.conf[detect].cpu()
                det['category'] = result.boxes.cls[detect].cpu()
                coords = result.boxes.xywhn[detect].cpu()
                det['bbox'] = [coords[0]-coords[2]/2, coords[1]-coords[3]/2, coords[2], coords[3]]
                detection_boxes.append(det)
                
            im_path = os.path.join(img_path + subdataset + '/images/', img)
            visutils.draw_bounding_boxes_on_file(im_path, save_path, detection_boxes,
                                            confidence_threshold=0.0, detector_label_map=None,
                                            thickness=1,expansion=0, colormap=['Red'])

            selected_label = img_path  + subdataset + '/labels/' + os.path.basename(result.path).split('.jpg')[0] + '.txt'
            if os.path.exists(selected_label):
                detection_boxes = []
                df = pd.read_csv(selected_label, sep='\t', header=None, index_col=False)
                for irow, row in df.iterrows():  
                    det = {}
                    det['conf'] = None
                    det['category'] = row[0]
                    det['bbox'] = [row[1]-row[3]/2, row[2]-row[4]/2, row[3], row[4]]
                    detection_boxes.append(det)
        
                # Draw annotations
                save_path2 = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/' + TASK + '/' + model_name + '/prediction_label_' + os.path.basename(result.path).split('.hpg')[0] + '.jpg'
                visutils.draw_bounding_boxes_on_file(save_path, save_path2, detection_boxes,
                                                confidence_threshold=0.0, detector_label_map=None,
                                                thickness=1,expansion=0, colormap=['SpringGreen'])
                
                # Remove predictions-only images
                os.remove(save_path)


In [ ]:
b

## YOLO Evaluation

In [ ]:
# EVALUATE the model's performance on the test set
metrics = model.val(split='test', save_json=True, iou=0.1, conf=0.2, max_det=600)

# Export the model to ONNX format
#success = model.export(format='onnx')

In [ ]:
print(metrics.box.map)
print(metrics.box.map50)    # map50-95

## YOLO Evaluation per dataset

In [ ]:
datasets =  ['global_birds_poland', 'global_birds_palmyra', 'global_birds_penguins',
                    'global_birds_mckellar', 'global_birds_newmexico', 
                    'global_birds_pfeifer', 'uav_thermal_waterfowl']

In [ ]:
model_name = 'pfeifer_penguins_poland_palmyra_mckellar_yolov8m_120epoch'
model = YOLO('runs/detect/' + model_name + '/weights/best.pt')

In [ ]:
for dataset in datasets:

    # Change test folder
    fname = "data.yaml"
    stream = open(fname, 'r')
    data = yaml.safe_load(stream)
    data['test'] = 'test/' + dataset + '/images/'
    with open(fname, 'w') as yaml_file:
        yaml_file.write( yaml.dump(data, default_flow_style=False))
    
    metrics = model.val(split='test', save_json=True, iou=0.1, max_det=600)
    print(metrics.box.map50)

# Change test folder to original name
fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
data['test'] = 'test/images/'

## **TEST**

In [ ]:
model_name = 'pfeifer_yolov8m_120epoch_default_batch32_aug90deg0.5flips_patience50_lr00.001_lrf0.0001'
model = YOLO('runs/detect/' + model_name + '/weights/best.pt')

In [ ]:
fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
img_path = data['path'] + '/test/'

selected_img = (random.choices(os.listdir(img_path + '/images/'), k=1))
selected_img = [os.path.join(img_path + '/images/', x) for x in selected_img]
selected_img

In [ ]:
output = model(selected_img)

In [ ]:
output